In [1]:
!pip install bs4

In [2]:
!pip install lxml

In [3]:
import requests
import pandas as pd
import time
import json
import logging
from bs4 import BeautifulSoup
import re
from requests.exceptions import RequestException

**Настройка логирования**

In [7]:
logging.basicConfig(filename='log_tripadvisor.log', level=logging.INFO, format='%(asctime)s   %(levelname)s   %(message)s', force=True)
logging.info('Тестовая запись')

**Endpoint 1: Поиск ID города**

In [ ]:
url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/search"

querystring = {"query":"london"}

headers = {
  #"x-rapidapi-key": "85c5187b10mshe8a7b6dfadf750fp11e8bejsn944c7ad11a9f",
  #"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response_0 = requests.get(url, headers=headers, params=querystring)

In [ ]:
response_0.json()

[{'id': 186338,
  'name': 'London',
  'link': 'https://www.tripadvisor.com/Restaurants-g186338-London_England.html',
  'type': 'city',
  'thumbnail_url': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/33/f5/de/london.jpg',
  'latitude': 51.51924,
  'longitude': -0.096654},
 {'id': 154995,
  'name': 'London',
  'link': 'https://www.tripadvisor.com/Restaurants-g154995-London_Ontario.html',
  'type': 'municipality',
  'thumbnail_url': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/01/7d/20/5e/london.jpg',
  'latitude': 42.988914,
  'longitude': -81.2442},
 {'id': 19382775,
  'name': 'The London',
  'link': 'https://www.tripadvisor.com/Restaurant_Review-g188636-d19382775-Reviews-The_London-Antwerp_Antwerp_Province.html',
  'type': 'eatery',
  'thumbnail_url': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/8c/50/2f/restaurant-the-london.jpg',
  'latitude': 51.231045,
  'longitude': 4.40619},
 {'id': 39598,
  'name': 'London',
  'link': 'https://www.tripad

Нужный ID для Лондона (Великобритания) - **186338**

Теперь необходимо выяснить, сколько страниц есть по ресторанам Лондона

In [32]:
url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"

querystring = {"query":"186338","page":"1"}

headers = {
	#"x-rapidapi-key": "85c5187b10mshe8a7b6dfadf750fp11e8bejsn944c7ad11a9f",
	#"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response_london = requests.get(url, headers=headers, params=querystring)
page_london = response_london.json()

In [29]:
page_london.keys()

dict_keys(['link', 'total_pages', 'current_page', 'total_items_count', 'items_count', 'results'])

In [30]:
page_london['current_page'], page_london['total_pages'], page_london['total_items_count'], page_london['items_count'], page_london['link'], len(page_london['results'])

(1,
 333,
 10000,
 35,
 'https://www.tripadvisor.com/Restaurants-g186338-London_England.html',
 35)

`current_page` - текущая страница: 1

`total_pages` - всего страниц: **333**

`total_items_count` - всего ресторанов: **10 000**

`items_count` - ресторанов на странице: 35

`link` - ссылка на страницу с ресторанами Лондона

`results` - информация о ресторанах

**Endpoint 2: Поиск ресторанов по ID города** (состоит из 2 частей из-за лимита запросов к API)

Часть 1

In [8]:
url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"

headers = {
    #"x-rapidapi-key": "ef9a43cef7msh0f8d4b01af7b27ep1b6605jsn29e299987492",
    #"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

# Создаем пустой DataFrame
df_restaurants = pd.DataFrame()

# Проходим по каждой странице с 1 по 160
for page_num in range(1, 161):
    # Обновляем параметр запроса для текущей страницы
    querystring = {"query": "186338", "page": str(page_num)}

    # Делаем запрос
    response = requests.get(url, headers=headers, params=querystring)

    # Если запрос прошел успешно (код ответа 200)
    if response.status_code == 200:
        # Преобразуем данные в JSON
        page_data = response.json()

        # Если на странице есть данные
        if page_data:
            # Преобразуем данные в DataFrame
            page_df = pd.DataFrame(page_data['results'])

            # Добавляем данные со страницы в основной DataFrame
            df_restaurants = pd.concat([df_restaurants, page_df], ignore_index=True)
            print(f"Страница {page_num} добавлена")

            # Выводим прогресс
            logging.info(f"Страница {page_num} обработана")
        else:
            logging.info(f"Нет объектов на странице {page_num}")
    else:
        logging.error(f"Ошибка на странице {page_num}, status code: {response.status_code}")

    # Делаем небольшую паузу между запросами, чтобы избежать перегрузки API
    time.sleep(1)

# Выводим итоговый DataFrame
df_restaurants

Страница 1 добавлена
Страница 2 добавлена
Страница 3 добавлена
Страница 4 добавлена
Страница 5 добавлена
Страница 6 добавлена
Страница 7 добавлена
Страница 8 добавлена
Страница 9 добавлена
Страница 10 добавлена
Страница 11 добавлена
Страница 12 добавлена
Страница 13 добавлена
Страница 14 добавлена
Страница 15 добавлена
Страница 16 добавлена
Страница 17 добавлена
Страница 18 добавлена
Страница 19 добавлена
Страница 20 добавлена
Страница 21 добавлена
Страница 22 добавлена
Страница 23 добавлена
Страница 24 добавлена
Страница 25 добавлена
Страница 26 добавлена
Страница 27 добавлена
Страница 28 добавлена
Страница 29 добавлена
Страница 30 добавлена
Страница 31 добавлена
Страница 32 добавлена
Страница 33 добавлена
Страница 34 добавлена
Страница 35 добавлена
Страница 36 добавлена
Страница 37 добавлена
Страница 38 добавлена
Страница 39 добавлена
Страница 40 добавлена
Страница 41 добавлена
Страница 42 добавлена
Страница 43 добавлена
Страница 44 добавлена
Страница 45 добавлена
Страница 46 добавле

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines
0,2418348,Leicester Arms,https://www.tripadvisor.com/Restaurant_Review-...,602,4.5,$$ - $$$,True,https://www.greeneking.co.uk/pubs/greater-lond...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.510597,-0.137677,False,False,"[British, Pub]"
1,26452946,Swiss Butter,https://www.tripadvisor.com/Restaurant_Review-...,182,4.0,$$ - $$$,True,https://swissbutter.com/uk,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520620,-0.122808,False,True,"[French, Steakhouse]"
2,25151198,Florattica Rooftop Bar London,https://www.tripadvisor.com/Restaurant_Review-...,316,4.5,$$ - $$$,False,https://floratticalondon.co.uk/menus/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.512955,-0.075292,False,True,"[Bar, European, British, Healthy, Dining bars]"
3,2213156,The Black Dog Freehouse,https://www.tripadvisor.com/Restaurant_Review-...,733,4.5,$$ - $$$,False,https://theblackdogvauxhall.co.uk/menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.488724,-0.120889,False,False,"[Bar, British, Pub]"
4,27930238,Fitz's Brasserie,https://www.tripadvisor.com/Restaurant_Review-...,48,5.0,$$ - $$$,False,https://www.kimptonfitzroylondon.com/wp-conten...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.522408,-0.124933,False,False,"[Steakhouse, British, Contemporary]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5588,1388958,Devonshire Terrace,https://www.tripadvisor.com/Restaurant_Review-...,240,3.5,$$ - $$$,False,https://www.drakeandmorgan.co.uk/devonshire-te...,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.516956,-0.078353,True,False,"[Bar, International]"
5589,1389564,Lord's Tavern,https://www.tripadvisor.com/Restaurant_Review-...,259,4.0,$$ - $$$,False,https://lords-stg.azureedge.net/mediafiles/lor...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.528057,-0.173285,False,False,[British]
5590,9784345,Kaffeine,https://www.tripadvisor.com/Restaurant_Review-...,71,4.5,$,False,https://kaffeine.co.uk/location/66-great-titch...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.516834,-0.137191,False,False,[Cafe]
5591,19100903,The Corn Exchange,https://www.tripadvisor.com/Restaurant_Review-...,40,4.5,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.510967,-0.080329,False,False,[British]


In [12]:
#df_restaurants.to_csv('df_restaurants_p_1.csv', index=False)

Часть 2

In [13]:
url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"

headers = {
    #"x-rapidapi-key": "36077ca6e6msh69228b64910c90fp17d035jsn5c7328f60bd4",
    #"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

# Проходим по каждой странице с 161 по 333
for page_num in range(161, 334):
    # Обновляем параметр запроса для текущей страницы
    querystring = {"query": "186338", "page": str(page_num)}

    # Делаем запрос
    response = requests.get(url, headers=headers, params=querystring)

    # Если запрос прошел успешно (код ответа 200)
    if response.status_code == 200:
        # Преобразуем данные в JSON
        page_data = response.json()

        # Если на странице есть данные
        if page_data:
            # Преобразуем данные в DataFrame
            page_df = pd.DataFrame(page_data['results'])

            # Добавляем данные со страницы в основной DataFrame
            df_restaurants = pd.concat([df_restaurants, page_df], ignore_index=True)
            print(f"Страница {page_num} добавлена")

            # Выводим прогресс
            logging.info(f"Страница {page_num} обработана")
        else:
            logging.info(f"Нет объектов на странице {page_num}")
    else:
        logging.error(f"Ошибка на странице {page_num}, status code: {response.status_code}")

    # Делаем небольшую паузу между запросами, чтобы избежать перегрузки API
    time.sleep(1)

# Выводим итоговый DataFrame
df_restaurants

Страница 161 добавлена
Страница 162 добавлена
Страница 163 добавлена
Страница 164 добавлена
Страница 165 добавлена
Страница 166 добавлена
Страница 167 добавлена
Страница 168 добавлена
Страница 169 добавлена
Страница 170 добавлена
Страница 171 добавлена
Страница 172 добавлена
Страница 173 добавлена
Страница 174 добавлена
Страница 175 добавлена
Страница 176 добавлена
Страница 177 добавлена
Страница 178 добавлена
Страница 179 добавлена
Страница 180 добавлена
Страница 181 добавлена
Страница 182 добавлена
Страница 183 добавлена
Страница 184 добавлена
Страница 185 добавлена
Страница 186 добавлена
Страница 187 добавлена
Страница 188 добавлена
Страница 189 добавлена
Страница 190 добавлена
Страница 191 добавлена
Страница 192 добавлена
Страница 193 добавлена
Страница 194 добавлена
Страница 195 добавлена
Страница 196 добавлена
Страница 197 добавлена
Страница 198 добавлена
Страница 199 добавлена
Страница 200 добавлена
Страница 201 добавлена
Страница 202 добавлена
Страница 203 добавлена
Страница 20

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines
0,2418348,Leicester Arms,https://www.tripadvisor.com/Restaurant_Review-...,602,4.5,$$ - $$$,True,https://www.greeneking.co.uk/pubs/greater-lond...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.510597,-0.137677,False,False,"[British, Pub]"
1,26452946,Swiss Butter,https://www.tripadvisor.com/Restaurant_Review-...,182,4.0,$$ - $$$,True,https://swissbutter.com/uk,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520620,-0.122808,False,True,"[French, Steakhouse]"
2,25151198,Florattica Rooftop Bar London,https://www.tripadvisor.com/Restaurant_Review-...,316,4.5,$$ - $$$,False,https://floratticalondon.co.uk/menus/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.512955,-0.075292,False,True,"[Bar, European, British, Healthy, Dining bars]"
3,2213156,The Black Dog Freehouse,https://www.tripadvisor.com/Restaurant_Review-...,733,4.5,$$ - $$$,False,https://theblackdogvauxhall.co.uk/menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.488724,-0.120889,False,False,"[Bar, British, Pub]"
4,27930238,Fitz's Brasserie,https://www.tripadvisor.com/Restaurant_Review-...,48,5.0,$$ - $$$,False,https://www.kimptonfitzroylondon.com/wp-conten...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.522408,-0.124933,False,False,"[Steakhouse, British, Contemporary]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11643,1059321,Kosmopol,https://www.tripadvisor.com/Restaurant_Review-...,40,4.0,$$ - $$$,False,https://www.kosmopollondon.co.uk/cocktail-menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.487255,-0.178910,False,False,[Bar]
11644,1803675,Nando's Camberwell,https://www.tripadvisor.com/Restaurant_Review-...,64,3.5,$$ - $$$,False,https://bit.ly/3DdhmAa,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.471750,-0.093243,False,False,"[Barbecue, Grill, Portuguese]"
11645,12999923,Sunnisa Thai Restaurant,https://www.tripadvisor.com/Restaurant_Review-...,28,4.5,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.464000,-0.269890,False,False,"[Asian, Thai]"
11646,6354369,Esters,https://www.tripadvisor.com/Restaurant_Review-...,42,4.5,$$ - $$$,False,http://www.estersn16.com/s/MENU_MAY_190529_B.docx,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.560173,-0.078153,False,False,"[Cafe, British, Fusion]"


In [14]:
#df_restaurants.to_csv('df_restaurants_full.csv', index=False)

In [17]:
df_restaurants['id'].duplicated().sum()

1658

In [19]:
df_restaurants_non_dupl = df_restaurants.drop_duplicates(['id'])
df_restaurants_non_dupl.reset_index(drop=True, inplace=True)
df_restaurants_non_dupl

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines
0,2418348,Leicester Arms,https://www.tripadvisor.com/Restaurant_Review-...,602,4.5,$$ - $$$,True,https://www.greeneking.co.uk/pubs/greater-lond...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.510597,-0.137677,False,False,"[British, Pub]"
1,26452946,Swiss Butter,https://www.tripadvisor.com/Restaurant_Review-...,182,4.0,$$ - $$$,True,https://swissbutter.com/uk,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520620,-0.122808,False,True,"[French, Steakhouse]"
2,25151198,Florattica Rooftop Bar London,https://www.tripadvisor.com/Restaurant_Review-...,316,4.5,$$ - $$$,False,https://floratticalondon.co.uk/menus/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.512955,-0.075292,False,True,"[Bar, European, British, Healthy, Dining bars]"
3,2213156,The Black Dog Freehouse,https://www.tripadvisor.com/Restaurant_Review-...,733,4.5,$$ - $$$,False,https://theblackdogvauxhall.co.uk/menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.488724,-0.120889,False,False,"[Bar, British, Pub]"
4,27930238,Fitz's Brasserie,https://www.tripadvisor.com/Restaurant_Review-...,48,5.0,$$ - $$$,False,https://www.kimptonfitzroylondon.com/wp-conten...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.522408,-0.124933,False,False,"[Steakhouse, British, Contemporary]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,7273839,Cafe de Pierre,https://www.tripadvisor.com/Restaurant_Review-...,152,3.5,$$ - $$$,False,http://www.caffeconcerto.co.uk/menus/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507324,-0.141801,False,False,"[Cafe, European]"
9986,1059321,Kosmopol,https://www.tripadvisor.com/Restaurant_Review-...,40,4.0,$$ - $$$,False,https://www.kosmopollondon.co.uk/cocktail-menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.487255,-0.178910,False,False,[Bar]
9987,1803675,Nando's Camberwell,https://www.tripadvisor.com/Restaurant_Review-...,64,3.5,$$ - $$$,False,https://bit.ly/3DdhmAa,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.471750,-0.093243,False,False,"[Barbecue, Grill, Portuguese]"
9988,12999923,Sunnisa Thai Restaurant,https://www.tripadvisor.com/Restaurant_Review-...,28,4.5,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.464000,-0.269890,False,False,"[Asian, Thai]"


Не хватает 10 ресторанов до 10 тыс. -> берем город рядом с Лондоном и запрашиваем еще рестораны с первой страницы

In [20]:
url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"

querystring = {"query":"slough","page":"1"}

headers = {
	#"x-rapidapi-key": "85c5187b10mshe8a7b6dfadf750fp11e8bejsn944c7ad11a9f",
	#"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response_sl = requests.get(url, headers=headers, params=querystring)
page_sl = response_sl.json()
df_slough_rest = pd.DataFrame(page_sl['results'])
df_slough_rest

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines
0,23939630,Amchur Restaurant And Bar,https://www.tripadvisor.com/Restaurant_Review-...,580,5.0,$$ - $$$,True,https://amchur.co.uk/dine-in-menu/,https://tripadvisor.com/Commerce?p=theFork-Laf...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.462370,-0.585811,True,False,"[Indian, Asian, Nepali]"
1,2707374,Earl of Cornwall,https://www.tripadvisor.com/Restaurant_Review-...,293,4.5,$,True,https://www.greeneking.co.uk/pubs/berkshire/ea...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.511967,-0.624003,False,False,"[Bar, British, Pub]"
2,7786363,Steakout,https://www.tripadvisor.com/Restaurant_Review-...,805,4.5,$$ - $$$,False,https://steakoutslough.com/order-now,https://tripadvisor.com/Commerce?p=Delivery_Ta...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507640,-0.586037,True,False,"[Steakhouse, British, Grill]"
3,2588015,Kings Head,https://www.tripadvisor.com/Restaurant_Review-...,275,4.5,$,False,https://www.greeneking.co.uk/pubs/berkshire/ki...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.514400,-0.639710,False,False,"[Bar, British, Grill, Pub]"
4,7067655,Panjab Restaurant,https://www.tripadvisor.com/Restaurant_Review-...,605,4.5,$$ - $$$,False,http://www.panjabrestaurant.co.uk/wp-content/u...,https://tripadvisor.com/Commerce?p=Delivery_Ta...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.500290,-0.564033,True,False,[Indian]
5,19084958,Renaizance,https://www.tripadvisor.com/Restaurant_Review-...,194,4.5,$$ - $$$,False,https://www.therenaizance.co.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520893,-0.625102,True,False,[Indian]
6,21227947,Akaya Restruant & Lounge (Halal),https://www.tripadvisor.com/Restaurant_Review-...,158,4.5,$$ - $$$,False,None,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507484,-0.585915,True,False,"[Indian, Mediterranean, Asian, Middle Eastern,..."
7,4475460,Javitri,https://www.tripadvisor.com/Restaurant_Review-...,400,4.5,$$ - $$$,True,https://javitri.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.549248,-0.482457,True,False,"[Indian, Asian, Balti, Bangladeshi]"
8,2707374,Earl of Cornwall,https://www.tripadvisor.com/Restaurant_Review-...,293,4.5,$,False,https://www.greeneking.co.uk/pubs/berkshire/ea...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.511967,-0.624003,False,False,"[Bar, British, Pub]"
9,32783208,Mova Group,https://www.tripadvisor.com/Restaurant_Review-...,14,5.0,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.509490,-0.595410,False,False,"[Chinese, Indian, Bar, Asian, Pub, Sri Lankan]"


In [24]:
df_slough_rest_non_dupl = df_slough_rest.drop_duplicates(['id'])
df_slough_rest_non_dupl.reset_index(drop=True, inplace=True)
df_slough_rest_non_dupl

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines
0,23939630,Amchur Restaurant And Bar,https://www.tripadvisor.com/Restaurant_Review-...,580,5.0,$$ - $$$,True,https://amchur.co.uk/dine-in-menu/,https://tripadvisor.com/Commerce?p=theFork-Laf...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.462370,-0.585811,True,False,"[Indian, Asian, Nepali]"
1,2707374,Earl of Cornwall,https://www.tripadvisor.com/Restaurant_Review-...,293,4.5,$,True,https://www.greeneking.co.uk/pubs/berkshire/ea...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.511967,-0.624003,False,False,"[Bar, British, Pub]"
2,7786363,Steakout,https://www.tripadvisor.com/Restaurant_Review-...,805,4.5,$$ - $$$,False,https://steakoutslough.com/order-now,https://tripadvisor.com/Commerce?p=Delivery_Ta...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507640,-0.586037,True,False,"[Steakhouse, British, Grill]"
3,2588015,Kings Head,https://www.tripadvisor.com/Restaurant_Review-...,275,4.5,$,False,https://www.greeneking.co.uk/pubs/berkshire/ki...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.514400,-0.639710,False,False,"[Bar, British, Grill, Pub]"
4,7067655,Panjab Restaurant,https://www.tripadvisor.com/Restaurant_Review-...,605,4.5,$$ - $$$,False,http://www.panjabrestaurant.co.uk/wp-content/u...,https://tripadvisor.com/Commerce?p=Delivery_Ta...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.500290,-0.564033,True,False,[Indian]
5,19084958,Renaizance,https://www.tripadvisor.com/Restaurant_Review-...,194,4.5,$$ - $$$,False,https://www.therenaizance.co.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520893,-0.625102,True,False,[Indian]
6,21227947,Akaya Restruant & Lounge (Halal),https://www.tripadvisor.com/Restaurant_Review-...,158,4.5,$$ - $$$,False,None,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507484,-0.585915,True,False,"[Indian, Mediterranean, Asian, Middle Eastern,..."
7,4475460,Javitri,https://www.tripadvisor.com/Restaurant_Review-...,400,4.5,$$ - $$$,True,https://javitri.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.549248,-0.482457,True,False,"[Indian, Asian, Balti, Bangladeshi]"
8,32783208,Mova Group,https://www.tripadvisor.com/Restaurant_Review-...,14,5.0,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.509490,-0.595410,False,False,"[Chinese, Indian, Bar, Asian, Pub, Sri Lankan]"
9,1553245,New Bettola,https://www.tripadvisor.com/Restaurant_Review-...,431,4.5,$$ - $$$,False,https://newbettola.co.uk/menu/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.479280,-0.511472,False,False,[Italian]


Соединяем с предыдущим датафреймом (London + Slough)

In [25]:
df_restaurants_full = pd.concat([df_restaurants_non_dupl, df_slough_rest_non_dupl], ignore_index=True)
df_restaurants_full

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines
0,2418348,Leicester Arms,https://www.tripadvisor.com/Restaurant_Review-...,602,4.5,$$ - $$$,True,https://www.greeneking.co.uk/pubs/greater-lond...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.510597,-0.137677,False,False,"[British, Pub]"
1,26452946,Swiss Butter,https://www.tripadvisor.com/Restaurant_Review-...,182,4.0,$$ - $$$,True,https://swissbutter.com/uk,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520620,-0.122808,False,True,"[French, Steakhouse]"
2,25151198,Florattica Rooftop Bar London,https://www.tripadvisor.com/Restaurant_Review-...,316,4.5,$$ - $$$,False,https://floratticalondon.co.uk/menus/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.512955,-0.075292,False,True,"[Bar, European, British, Healthy, Dining bars]"
3,2213156,The Black Dog Freehouse,https://www.tripadvisor.com/Restaurant_Review-...,733,4.5,$$ - $$$,False,https://theblackdogvauxhall.co.uk/menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.488724,-0.120889,False,False,"[Bar, British, Pub]"
4,27930238,Fitz's Brasserie,https://www.tripadvisor.com/Restaurant_Review-...,48,5.0,$$ - $$$,False,https://www.kimptonfitzroylondon.com/wp-conten...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.522408,-0.124933,False,False,"[Steakhouse, British, Contemporary]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10017,17306111,Damascene,https://www.tripadvisor.com/Restaurant_Review-...,46,4.0,$,False,None,https://tripadvisor.com/Commerce?p=Delivery_Ta...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507507,-0.587769,True,False,"[Lebanese, Mediterranean, Barbecue, Grill, Mid..."
10018,15658775,Swagath,https://www.tripadvisor.com/Restaurant_Review-...,83,4.0,$$ - $$$,False,https://swagathslough.co.uk/menu/,https://tripadvisor.com/Commerce?p=Delivery_Ta...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.509247,-0.594417,True,True,"[Indian, Asian, Street Food]"
10019,19947441,Amrita's Kitchen,https://www.tripadvisor.com/Restaurant_Review-...,18,5.0,$,False,https://drive.google.com/file/d/1qNgddi7n2KzAf...,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.488460,-0.534572,True,False,"[Indian, Cafe, Asian, Healthy, Street Food]"
10020,10133979,The Crown & Kitchen,https://www.tripadvisor.com/Restaurant_Review-...,34,4.5,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.473600,-0.543492,False,False,"[Indian, Bar, Pub]"


In [26]:
df_restaurants_full['id'].duplicated().sum()

0

In [33]:
df_restaurants_10k = df_restaurants_full.head(10000)
df_restaurants_10k

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines
0,2418348,Leicester Arms,https://www.tripadvisor.com/Restaurant_Review-...,602,4.5,$$ - $$$,True,https://www.greeneking.co.uk/pubs/greater-lond...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.510597,-0.137677,False,False,"[British, Pub]"
1,26452946,Swiss Butter,https://www.tripadvisor.com/Restaurant_Review-...,182,4.0,$$ - $$$,True,https://swissbutter.com/uk,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520620,-0.122808,False,True,"[French, Steakhouse]"
2,25151198,Florattica Rooftop Bar London,https://www.tripadvisor.com/Restaurant_Review-...,316,4.5,$$ - $$$,False,https://floratticalondon.co.uk/menus/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.512955,-0.075292,False,True,"[Bar, European, British, Healthy, Dining bars]"
3,2213156,The Black Dog Freehouse,https://www.tripadvisor.com/Restaurant_Review-...,733,4.5,$$ - $$$,False,https://theblackdogvauxhall.co.uk/menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.488724,-0.120889,False,False,"[Bar, British, Pub]"
4,27930238,Fitz's Brasserie,https://www.tripadvisor.com/Restaurant_Review-...,48,5.0,$$ - $$$,False,https://www.kimptonfitzroylondon.com/wp-conten...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.522408,-0.124933,False,False,"[Steakhouse, British, Contemporary]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19084958,Renaizance,https://www.tripadvisor.com/Restaurant_Review-...,194,4.5,$$ - $$$,False,https://www.therenaizance.co.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520893,-0.625102,True,False,[Indian]
9996,21227947,Akaya Restruant & Lounge (Halal),https://www.tripadvisor.com/Restaurant_Review-...,158,4.5,$$ - $$$,False,None,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507484,-0.585915,True,False,"[Indian, Mediterranean, Asian, Middle Eastern,..."
9997,4475460,Javitri,https://www.tripadvisor.com/Restaurant_Review-...,400,4.5,$$ - $$$,True,https://javitri.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.549248,-0.482457,True,False,"[Indian, Asian, Balti, Bangladeshi]"
9998,32783208,Mova Group,https://www.tripadvisor.com/Restaurant_Review-...,14,5.0,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.509490,-0.595410,False,False,"[Chinese, Indian, Bar, Asian, Pub, Sri Lankan]"


In [34]:
#df_restaurants_10k.to_csv('df_restaurants_10k.csv', index=False)

**Endpoint 3: Поиск деталей ресторанов по ID городов**

In [35]:
# Пустой датафрейм для хранения данных о ресторанах
restaurant_details_df = pd.DataFrame()

# Цикл по каждому ресторану из столбца id
for rest_id in df_restaurants_10k['id']:
    url = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/detail"

    querystring = {"id": str(rest_id)}

    headers = {
        #"x-rapidapi-key": "91dac91d71msh34e36e8dd740994p16ff5djsnccc90a15c067",
        #"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
    }

    # Отправляем запрос и получаем данные
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        # Преобразуем ответ в JSON
        rest_info = response.json()

        # Преобразуем данные ресторана в датафрейм и добавляем его к основному
        temp_df = pd.DataFrame([rest_info])
        restaurant_details_df = pd.concat([restaurant_details_df, temp_df], ignore_index=True)
        logging.info(f"Добавлены данные по ресторану {rest_id}")
        print(f"Добавлены данные по ресторану {rest_id}")
    else:
        logging.error(f"Ошибка при запросе ID {rest_id}: {response.status_code}")
    time.sleep(1)

restaurant_details_df

Добавлены данные по ресторану 2418348
Добавлены данные по ресторану 26452946
Добавлены данные по ресторану 25151198
Добавлены данные по ресторану 2213156
Добавлены данные по ресторану 27930238
Добавлены данные по ресторану 28156752
Добавлены данные по ресторану 17581691
Добавлены данные по ресторану 8565720
Добавлены данные по ресторану 25336907
Добавлены данные по ресторану 25268623
Добавлены данные по ресторану 12527182
Добавлены данные по ресторану 8802890
Добавлены данные по ресторану 32796367
Добавлены данные по ресторану 776287
Добавлены данные по ресторану 11856169
Добавлены данные по ресторану 7345750
Добавлены данные по ресторану 28254969
Добавлены данные по ресторану 719681
Добавлены данные по ресторану 13831655
Добавлены данные по ресторану 13377811
Добавлены данные по ресторану 26318360
Добавлены данные по ресторану 2011025
Добавлены данные по ресторану 10464803
Добавлены данные по ресторану 5247729
Добавлены данные по ресторану 2430068
Добавлены данные по ресторану 2116638

<ipython-input-35-92f70ad9b0d3>:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  restaurant_details_df = pd.concat([restaurant_details_df, temp_df], ignore_index=True)


Выходные данные были обрезаны до нескольких последних строк (5000).
Добавлены данные по ресторану 13525881
Добавлены данные по ресторану 1016845
Добавлены данные по ресторану 967803
Добавлены данные по ресторану 4893952
Добавлены данные по ресторану 20265053
Добавлены данные по ресторану 8511086
Добавлены данные по ресторану 3663396
Добавлены данные по ресторану 14940020
Добавлены данные по ресторану 1015005
Добавлены данные по ресторану 1436568
Добавлены данные по ресторану 15698034
Добавлены данные по ресторану 809826
Добавлены данные по ресторану 12079873
Добавлены данные по ресторану 10488459
Добавлены данные по ресторану 23705991
Добавлены данные по ресторану 720322
Добавлены данные по ресторану 25467511
Добавлены данные по ресторану 26524213
Добавлены данные по ресторану 25280611
Добавлены данные по ресторану 1318390
Добавлены данные по ресторану 2186638
Добавлены данные по ресторану 5769001
Добавлены данные по ресторану 7747061
Добавлены данные по ресторану 17189105
Добавлены да

,id,name,description,reviews,rating,link,email,phone,website,featured_image,...,open_hours,menu_link,delivery_url,price_range,cuisines,diets,meal_types,dining_options,owner_types,top_tags
0,2418348,Leicester Arms,The Leicester Arms is a traditional 19th centu...,602,4.5,https://www.tripadvisor.com/Restaurant_Review-...,7213@greeneking.co.uk,+44 20 7734 7641,https://www.greeneking.co.uk/pubs/greater-lond...,https://media-cdn.tripadvisor.com/media/photo-...,...,"{'sun': [{'open': '11:00:00', 'close': '22:30:...",https://www.greeneking.co.uk/pubs/greater-lond...,None,$$ - $$$,"[British, Pub]","[Vegetarian friendly, Gluten free options]","[Lunch, Dinner, Drinks]","[Reservations, Seating, Street Parking, Televi...",[],"[Mid-range, British, Pub, Vegetarian friendly]"
1,26452946,Swiss Butter,Swiss Butter is a casual steakhouse that serve...,182,4.0,https://www.tripadvisor.com/Restaurant_Review-...,None,+44 20 3685 2741,https://swissbutter.com/,https://media-cdn.tripadvisor.com/media/photo-...,...,"{'sun': [{'open': '12:00:00', 'close': '23:00:...",https://swissbutter.com/uk,None,$$ - $$$,"[French, Steakhouse]",[],"[Lunch, Dinner]","[Seating, Wheelchair Accessible, Serves Alcoho...",[],"[Mid-range, French, Steakhouse]"
2,25151198,Florattica Rooftop Bar London,"Situated high above the city in the Minories, ...",316,4.5,https://www.tripadvisor.com/Restaurant_Review-...,info@florattica.com,+44 20 3988 4489,http://www.floratticalondon.co.uk,https://media-cdn.tripadvisor.com/media/photo-...,...,"{'sun': [{'open': '12:00:00', 'close': '23:00:...",https://floratticalondon.co.uk/menus/,None,$$ - $$$,"[Bar, European, British, Healthy, Dining bars]",[],"[Dinner, Brunch, Drinks]","[Reservations, Outdoor Seating, Seating, Serve...",[],"[Mid-range, Bar, European]"
3,2213156,The Black Dog Freehouse,None,733,4.5,https://www.tripadvisor.com/Restaurant_Review-...,bookings@theblackdogvauxhall.co.uk,+44 20 7735 4440,https://theblackdogvauxhall.co.uk/,https://media-cdn.tripadvisor.com/media/photo-...,...,"{'sun': [{'open': '12:00:00', 'close': '22:00:...",https://theblackdogvauxhall.co.uk/menu,None,$$ - $$$,"[Bar, British, Pub]","[Vegetarian friendly, Vegan options, Gluten fr...","[Lunch, Dinner, Brunch, Late Night, Drinks]","[Reservations, Outdoor Seating, Seating, Highc...",[],"[Mid-range, Bar, British, Vegetarian friendly]"
4,27930238,Fitz's Brasserie,Standing as a charming testament to the cheris...,48,5.0,https://www.tripadvisor.com/Restaurant_Review-...,restaurants@kimptonfitzroylondon.com,+44 20 7520 1920,https://www.kimptonfitzroylondon.com/fitzsbras...,https://media-cdn.tripadvisor.com/media/photo-...,...,"{'sun': [{'open': '07:30:00', 'close': '11:00:...",https://www.kimptonfitzroylondon.com/wp-conten...,None,$$ - $$$,"[Steakhouse, British, Contemporary]",[Vegetarian friendly],"[Breakfast, Lunch, Dinner]","[Reservations, Seating, Wheelchair Accessible,...",[],"[Mid-range, Steakhouse, British, Vegetarian fr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,19084958,Renaizance,The Renaizance AUTHENTIC INDIAN CUISINE Here a...,194,4.5,https://www.tripadvisor.com/Restaurant_Review-...,info@therenaizance.co.uk,+44 1753 376304,http://www.therenaizance.co.uk,https://media-cdn.tripadvisor.com/media/photo-...,...,"{'sun': [{'open': '10:00:00', 'close': '22:00:...",https://www.therenaizance.co.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,$$ - $$$,[Indian],"[Vegetarian friendly, Vegan options, Gluten fr...","[Lunch, Dinner, Brunch, Drinks]","[Takeout, Reservations, Seating, Parking Avail...",[],"[Mid-range, Indian, Vegetarian friendly]"
9994,21227947,Akaya Restruant & Lounge (Halal),Our high-end restaurants have become renowned ...,158,4.5,https://www.tripadvisor.com/Restaurant_Review-...,None,+44 1753 369123,https://www.akayalounge.co.uk/,https://media-cdn.tripadvisor.com/media/photo-...,...,"{'sun': [{'open': '11:30:00', 'close': '00:00:...",None,https://tripadvisor.com/Commerce?p=Restaurants..

In [36]:
#restaurant_details_df.to_csv('tripadvisor_restaurant_details.csv', index=False)

In [37]:
restaurant_details_df_act = restaurant_details_df[['id', 'reviews_per_rating', 'diets', 'dining_options']]
restaurant_details_df_act

,id,reviews_per_rating,diets,dining_options
0,2418348,"{'1': 30, '2': 23, '3': 51, '4': 156, '5': 342}","[Vegetarian friendly, Gluten free options]","[Reservations, Seating, Street Parking, Televi..."
1,26452946,"{'1': 21, '2': 15, '3': 14, '4': 22, '5': 110}",[],"[Seating, Wheelchair Accessible, Serves Alcoho..."
2,25151198,"{'1': 15, '2': 4, '3': 7, '4': 11, '5': 279}",[],"[Reservations, Outdoor Seating, Seating, Serve..."
3,2213156,"{'1': 19, '2': 23, '3': 47, '4': 216, '5': 428}","[Vegetarian friendly, Vegan options, Gluten fr...","[Reservations, Outdoor Seating, Seating, Highc..."
4,27930238,"{'1': 0, '2': 1, '3': 1, '4': 2, '5': 44}",[Vegetarian friendly],"[Reservations, Seating, Wheelchair Accessible,..."
...,...,...,...,...
9993,19084958,"{'1': 14, '2': 9, '3': 15, '4': 26, '5': 130}","[Vegetarian friendly, Vegan options, Gluten fr...","[Takeout, Reservations, Seating, Parking Avail..."
9994,21227947,"{'1': 22, '2': 4, '3': 4, '4': 4, '5': 124}",[],"[Reservations, Seating, Table Service]"
9995,4475460,"{'1': 22, '2': 17, '3': 34, '4': 83, '5': 244}","[Vegetarian friendly, Vegan options, Halal, Gl...","[Delivery, Takeout, Reservations, Seating, Str..."
9996,32783208,"{'1': 0, '2': 0, '3': 0, '4': 0, '5': 14}",[],"[Reservations, Seating, Table Service]"


In [38]:
merged_df_tripadvisor = pd.merge(df_restaurants_10k, restaurant_details_df_act, on='id')
merged_df_tripadvisor

,id,name,link,reviews,rating,price_range_usd,is_sponsored,menu_link,reservation_link,featured_image,latitude,longitude,has_delivery,is_premium,cuisines,reviews_per_rating,diets,dining_options
0,2418348,Leicester Arms,https://www.tripadvisor.com/Restaurant_Review-...,602,4.5,$$ - $$$,True,https://www.greeneking.co.uk/pubs/greater-lond...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.510597,-0.137677,False,False,"[British, Pub]","{'1': 30, '2': 23, '3': 51, '4': 156, '5': 342}","[Vegetarian friendly, Gluten free options]","[Reservations, Seating, Street Parking, Televi..."
1,26452946,Swiss Butter,https://www.tripadvisor.com/Restaurant_Review-...,182,4.0,$$ - $$$,True,https://swissbutter.com/uk,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520620,-0.122808,False,True,"[French, Steakhouse]","{'1': 21, '2': 15, '3': 14, '4': 22, '5': 110}",[],"[Seating, Wheelchair Accessible, Serves Alcoho..."
2,25151198,Florattica Rooftop Bar London,https://www.tripadvisor.com/Restaurant_Review-...,316,4.5,$$ - $$$,False,https://floratticalondon.co.uk/menus/,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.512955,-0.075292,False,True,"[Bar, European, British, Healthy, Dining bars]","{'1': 15, '2': 4, '3': 7, '4': 11, '5': 279}",[],"[Reservations, Outdoor Seating, Seating, Serve..."
3,2213156,The Black Dog Freehouse,https://www.tripadvisor.com/Restaurant_Review-...,733,4.5,$$ - $$$,False,https://theblackdogvauxhall.co.uk/menu,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.488724,-0.120889,False,False,"[Bar, British, Pub]","{'1': 19, '2': 23, '3': 47, '4': 216, '5': 428}","[Vegetarian friendly, Vegan options, Gluten fr...","[Reservations, Outdoor Seating, Seating, Highc..."
4,27930238,Fitz's Brasserie,https://www.tripadvisor.com/Restaurant_Review-...,48,5.0,$$ - $$$,False,https://www.kimptonfitzroylondon.com/wp-conten...,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.522408,-0.124933,False,False,"[Steakhouse, British, Contemporary]","{'1': 0, '2': 1, '3': 1, '4': 2, '5': 44}",[Vegetarian friendly],"[Reservations, Seating, Wheelchair Accessible,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,19084958,Renaizance,https://www.tripadvisor.com/Restaurant_Review-...,194,4.5,$$ - $$$,False,https://www.therenaizance.co.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.520893,-0.625102,True,False,[Indian],"{'1': 14, '2': 9, '3': 15, '4': 26, '5': 130}","[Vegetarian friendly, Vegan options, Gluten fr...","[Takeout, Reservations, Seating, Parking Avail..."
9994,21227947,Akaya Restruant & Lounge (Halal),https://www.tripadvisor.com/Restaurant_Review-...,158,4.5,$$ - $$$,False,None,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.507484,-0.585915,True,False,"[Indian, Mediterranean, Asian, Middle Eastern,...","{'1': 22, '2': 4, '3': 4, '4': 4, '5': 124}",[],"[Reservations, Seating, Table Service]"
9995,4475460,Javitri,https://www.tripadvisor.com/Restaurant_Review-...,400,4.5,$$ - $$$,True,https://javitri.uk/menu,https://tripadvisor.com/Commerce?p=Restaurants...,https://dynamic-media-cdn.tripadvisor.com/medi...,51.549248,-0.482457,True,False,"[Indian, Asian, Balti, Bangladeshi]","{'1': 22, '2': 17, '3': 34, '4': 83, '5': 244}","[Vegetarian friendly, Vegan options, Halal, Gl...","[Delivery, Takeout, Reservations, Seating, Str..."
9996,32783208,Mova Group,https://www.tripadvisor.com/Restaurant_Review-...,14,5.0,$$ - $$$,False,None,None,https://dynamic-media-cdn.tripadvisor.com/medi...,51.509490,-0.595410,False,False,"[Chinese, Indian, Bar, Asian, Pub, Sri Lankan]","{'1': 0, '2': 0, '3': 0, '4': 0, '5': 14}",[],"[Reservations, Seating, Table Service]"


In [39]:
merged_df_tripadvisor_act = merged_df_tripadvisor.drop(['link', 'menu_link', 'reservation_link', 'featured_image'], axis=1)
merged_df_tripadvisor_act

,id,name,reviews,rating,price_range_usd,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,reviews_per_rating,diets,dining_options
0,2418348,Leicester Arms,602,4.5,$$ - $$$,True,51.510597,-0.137677,False,False,"[British, Pub]","{'1': 30, '2': 23, '3': 51, '4': 156, '5': 342}","[Vegetarian friendly, Gluten free options]","[Reservations, Seating, Street Parking, Televi..."
1,26452946,Swiss Butter,182,4.0,$$ - $$$,True,51.520620,-0.122808,False,True,"[French, Steakhouse]","{'1': 21, '2': 15, '3': 14, '4': 22, '5': 110}",[],"[Seating, Wheelchair Accessible, Serves Alcoho..."
2,25151198,Florattica Rooftop Bar London,316,4.5,$$ - $$$,False,51.512955,-0.075292,False,True,"[Bar, European, British, Healthy, Dining bars]","{'1': 15, '2': 4, '3': 7, '4': 11, '5': 279}",[],"[Reservations, Outdoor Seating, Seating, Serve..."
3,2213156,The Black Dog Freehouse,733,4.5,$$ - $$$,False,51.488724,-0.120889,False,False,"[Bar, British, Pub]","{'1': 19, '2': 23, '3': 47, '4': 216, '5': 428}","[Vegetarian friendly, Vegan options, Gluten fr...","[Reservations, Outdoor Seating, Seating, Highc..."
4,27930238,Fitz's Brasserie,48,5.0,$$ - $$$,False,51.522408,-0.124933,False,False,"[Steakhouse, British, Contemporary]","{'1': 0, '2': 1, '3': 1, '4': 2, '5': 44}",[Vegetarian friendly],"[Reservations, Seating, Wheelchair Accessible,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,19084958,Renaizance,194,4.5,$$ - $$$,False,51.520893,-0.625102,True,False,[Indian],"{'1': 14, '2': 9, '3': 15, '4': 26, '5': 130}","[Vegetarian friendly, Vegan options, Gluten fr...","[Takeout, Reservations, Seating, Parking Avail..."
9994,21227947,Akaya Restruant & Lounge (Halal),158,4.5,$$ - $$$,False,51.507484,-0.585915,True,False,"[Indian, Mediterranean, Asian, Middle Eastern,...","{'1': 22, '2': 4, '3': 4, '4': 4, '5': 124}",[],"[Reservations, Seating, Table Service]"
9995,4475460,Javitri,400,4.5,$$ - $$$,True,51.549248,-0.482457,True,False,"[Indian, Asian, Balti, Bangladeshi]","{'1': 22, '2': 17, '3': 34, '4': 83, '5': 244}","[Vegetarian friendly, Vegan options, Halal, Gl...","[Delivery, Takeout, Reservations, Seating, Str..."
9996,32783208,Mova Group,14,5.0,$$ - $$$,False,51.509490,-0.595410,False,False,"[Chinese, Indian, Bar, Asian, Pub, Sri Lankan]","{'1': 0, '2': 0, '3': 0, '4': 0, '5': 14}",[],"[Reservations, Seating, Table Service]"


In [41]:
#merged_df_tripadvisor_act.to_csv('tripadvisor_data_rest.csv', index=False)